<a href="https://colab.research.google.com/github/mojtabaSefidi/Machine-Learning-with-Graphs/blob/main/MLG_Ex2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install Essential Packages

In [1]:
!pip install -q torch-scatter -f https://pytorch-geometric.com/whl/torch-1.7.0+cu101.html
!pip install -q torch-sparse -f https://pytorch-geometric.com/whl/torch-1.7.0+cu101.html
!pip install -q torch-geometric

     |████████████████████████████████| 2.1 MB 6.9 MB/s 
     |████████████████████████████████| 467 kB 6.8 MB/s 


## Install Essential Libraries

In [4]:
%matplotlib inline
import matplotlib.pyplot as plt
import torch
import networkx as nx
import torch_geometric
from torch_geometric.utils import to_networkx
from torch_geometric.datasets import Planetoid
from torch_geometric.transforms import NormalizeFeatures

## Read the Dataset

In [ ]:
dataset = Planetoid(root='/tmp/Citeseer', name='Citeseer')

In [5]:
dataset = Planetoid(root='data/Planetoid', name='Cora', transform=NormalizeFeatures())

Processing...
Done!


## Part 1 : Dataset Details

In [6]:
def Information_Extaction(dataset):
  print('There is %d graph in the dataset.' %len(dataset))
  print('The graph has %d nodes.' %dataset[0].num_nodes)
  print('The graph has %d edges.' %dataset[0].num_edges)
  print('Each node has %d features.' %dataset[0].num_features)
  print('These are %d classes.'  %dataset.num_classes)
  print("The Average degree for each node is %d." %round((2*dataset[0].num_edges) / (dataset[0].num_nodes),4))
  print('There are %d training nodes in the dataset.' %dataset[0].train_mask.sum().item())
  return

In [7]:
Information_Extaction(dataset)

There is 1 graph in the dataset.
The graph has 2708 nodes.
The graph has 10556 edges.
Each node has 1433 features.
These are 7 classes.
The Average degree for each node is 7.
There are 140 training nodes in the dataset.


In [ ]:
show_edges(dataset,node_index=1028)

tensor([[1028, 1557],
        [1028, 2320],
        [1028, 2605]])

## Part 4 : Graph visualization

In [11]:
# visualize_graph(dataset[0],50)

## Additional Libraries for GNN

In [98]:
import torch.nn.functional as F
from torch.nn import Linear
from torch_geometric.nn import GCNConv

## Part 5 : GCN Implementation

In [ ]:
# class GCN(torch.nn.Module):
#     def __init__(
#         self,
#         dataset,
#         hidden_channel=256,
#         learning_rate=0.01,
#         min_valid_loss = np.inf,
        
        
#         ):
      
#         super().__init__()
#         self.conv1 = GCNConv(dataset.num_node_features, hidden_channel)
#         self.conv2 = GCNConv(hidden_channel, hidden_channel)
#         self.classifier = Linear(hidden_channel, dataset.num_classes)
        
#         device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#         self.data = dataset[0].to(device)
#         self.optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
#         self.criterion = torch.nn.CrossEntropyLoss()
#         self.min_valid_loss = min_valid_loss

#     def forward(self, data):
#         x, edge_index = data.x, data.edge_index

#         x = self.conv1(x, edge_index)
#         x = torch.tanh(x)
#         x = F.dropout(x, p=0.5, training=self.training)
#         x = self.conv2(x, edge_index)
#         x = torch.tanh(x)
#         return self.classifier(x)

#     def training(self):
#           save_model_flag = False
#           model.train()
#           self.optimizer.zero_grad()
#           out = model(self.data)
#           train_loss = self.criterion(out[self.data.train_mask], self.data.y[self.data.train_mask])
#           validation_loss = self.criterion(out[self.data.val_mask], self.data.y[self.data.val_mask])
#           if self.min_valid_loss > validation_loss:
#               save_model_flag = True
#           train_loss.backward()
#           self.optimizer.step()

#           return train_loss, validation_loss, save_model_flag 

#     def test(self):
#           out = model(self.data)
#           test_loss = self.criterion(out[self.data.test_mask], self.data.y[self.data.test_mask])
#           pred = out.argmax(dim=1)
#           test_correct = pred[self.data.test_mask] == self.data.y[self.data.test_mask]
#           test_acc = int(test_correct.sum()) / int(self.data.test_mask.sum())
#           return test_acc, test_loss
    
#     def fit_optimize(self, epochs):
#       for epoch in range(1, epochs+1):
#         train_loss, validation_loss, save_model_flag = self.training()
#         print(f'Epoch: {epoch:03d}, Trian Loss: {train_loss:.4f}, Validation Loss: {validation_loss:.4f}')
#         if save_model_flag:
#           self.min_valid_loss = validation_loss
#           print(f"Best validation loss: {self.min_valid_loss:.4f}")
#           print(f"Saving best model for epoch: {epoch}")
#           torch.save(model,'best_model.pth')


In [198]:
class GCN(torch.nn.Module):
    def __init__(
        self,
        hidden_channel=256,
        learning_rate=0.01
        
        ):
      
        super().__init__()
        self.conv1 = GCNConv(dataset.num_node_features, hidden_channel)
        self.conv2 = GCNConv(hidden_channel, hidden_channel)
        self.conv3 = GCNConv(hidden_channel, dataset.num_classes)


    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = torch.tanh(x)
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        return x


In [199]:
model = GCN(hidden_channel = 16)
print(model)

GCN(
  (conv1): GCNConv(1433, 16)
  (conv2): GCNConv(16, 16)
  (conv3): GCNConv(16, 7)
)


In [200]:
import numpy as np
learning_rate = 0.01
min_valid_loss = np.inf
epochs=150

In [201]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data = dataset[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion = torch.nn.CrossEntropyLoss()

In [202]:
def train(min_valid_loss):
      save_model_flag = False
      model.train()
      optimizer.zero_grad()
      out = model(data)
      train_loss = criterion(out[data.train_mask], data.y[data.train_mask])
      validation_loss = criterion(out[data.val_mask], data.y[data.val_mask])
      if min_valid_loss > validation_loss:
          save_model_flag = True
      train_loss.backward()
      optimizer.step()

      return train_loss, validation_loss, save_model_flag 

def test():
      model.eval()
      out = model(data)
      test_loss = criterion(out[data.test_mask], data.y[data.test_mask])
      pred = out.argmax(dim=1)
      test_correct = pred[data.test_mask] == data.y[data.test_mask]
      test_acc = int(test_correct.sum()) / int(data.test_mask.sum())
      return test_acc, test_loss

## Test Performance

In [203]:
for epoch in range(1, epochs+1):
  train_loss, validation_loss, save_model_flag = train(min_valid_loss)
  print(f'Epoch: {epoch:03d}, Trian Loss: {train_loss:.4f}, Validation Loss: {validation_loss:.4f}')
  if save_model_flag:
    min_valid_loss = validation_loss
    print()
    print(f"Best validation loss: {min_valid_loss:.4f}")
    print(f"Saving best model for epoch: {epoch}")
    print()
    torch.save(model,'best_model.pth')

Epoch: 001, Trian Loss: 2.7715, Validation Loss: 2.7719

Best validation loss: 2.7719
Saving best model for epoch: 1

Epoch: 002, Trian Loss: 2.7437, Validation Loss: 2.7486

Best validation loss: 2.7486
Saving best model for epoch: 2

Epoch: 003, Trian Loss: 2.7120, Validation Loss: 2.7243

Best validation loss: 2.7243
Saving best model for epoch: 3

Epoch: 004, Trian Loss: 2.6773, Validation Loss: 2.6960

Best validation loss: 2.6960
Saving best model for epoch: 4

Epoch: 005, Trian Loss: 2.6335, Validation Loss: 2.6637

Best validation loss: 2.6637
Saving best model for epoch: 5

Epoch: 006, Trian Loss: 2.5911, Validation Loss: 2.6245

Best validation loss: 2.6245
Saving best model for epoch: 6

Epoch: 007, Trian Loss: 2.5456, Validation Loss: 2.5903

Best validation loss: 2.5903
Saving best model for epoch: 7

Epoch: 008, Trian Loss: 2.4934, Validation Loss: 2.5478

Best validation loss: 2.5478
Saving best model for epoch: 8

Epoch: 009, Trian Loss: 2.4419, Validation Loss: 2.5051


In [204]:
model = torch.load('best_model.pth')
test_acc, test_loss = test()
print(f'Test Accuracy: {test_acc:.4f}, Test loss: {test_loss:.4f}')

Test Accuracy: 0.7710, Test loss: 0.7501
